In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.model_selection import GridSearchCV

from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
%matplotlib inline

In [3]:
training_labels = pd.read_csv("set_entrenamiento.csv", low_memory=False)
labels_predict = pd.read_csv("trocafone_kaggle_test.csv", low_memory=False)
data = pd.read_csv("data_set.csv", low_memory = False)

In [4]:
training_labels = training_labels[['person','label']]
training_labels['label'].value_counts()

1    980
0    980
Name: label, dtype: int64

In [5]:
#data = data[['person','days_since_last_session','checkout','promedio dias','sessions','viewed/checkout gral',
#             'total_views', 'viewed product', 'promedio hora','retornos']]
data.head()

,person,Direct,Email,Organic,Paid,Referral,Social,ad campaign hit,brand listing,checkout,...,same_product_max_views,viewed/checkouted min,viewed/checkouted max,viewed/checkout gral,Computer,Smartphone,Tablet,country_Computer,country_Smartphone,country_Tablet
0,0008ed71,2.0,0.0,0.0,0.0,0.0,0.0,0,0,3,...,0.0,-1.0,-1.0,0.00,2.0,0.0,0.0,1.0,0.0,0.0
1,00091926,1.0,0.0,25.0,0.0,0.0,0.0,15,25,2,...,33.0,1.0,0.0,82.00,34.0,0.0,0.0,1.0,0.0,0.0
2,00091a7a,0.0,0.0,0.0,0.0,0.0,0.0,1,5,0,...,1.0,0.0,0.0,-1.00,0.0,1.0,0.0,0.0,1.0,0.0
3,000ba417,0.0,0.0,6.0,0.0,0.0,0.0,1,24,6,...,10.0,1.0,0.0,20.25,6.0,0.0,0.0,1.0,0.0,0.0
4,000c79fe,0.0,0.0,0.0,0.0,0.0,0.0,1,0,1,...,3.0,1.0,1.0,1.00,0.0,1.0,0.0,0.0,1.0,0.0


In [6]:
features = pd.merge(training_labels, data, on='person', how='inner')
features = features.fillna(0)
features['label'].value_counts()

1    980
0    980
Name: label, dtype: int64

In [7]:
labels = features['label']
features = features.drop(['label','person'], axis=1)
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.25)
print("Train: ",len(x_train),"Test: ",len(x_test))

Train:  1470 Test:  490


In [8]:
import catboost as cb
cat_features_index = [0,1,2,3,4,5,6]

In [9]:
def auc(m, train, test): 
    return (metrics.roc_auc_score(y_train,m.predict_proba(train)[:,1]),
                            metrics.roc_auc_score(y_test,m.predict_proba(test)[:,1]))

In [10]:
import catboost as cb
from catboost import CatBoostClassifier

params = {'depth': [4, 7, 10],
          'learning_rate' : [0.03, 0.1, 0.15],
         'l2_leaf_reg': [1,4,9],
         'iterations': [300]}
cb = CatBoostClassifier()
cb_model = GridSearchCV(cb, params, scoring="roc_auc", cv = 3)
cb_model.fit(x_train, y_train)

0:	learn: 0.6822867	total: 61.9ms	remaining: 18.5s
1:	learn: 0.6737259	total: 72.6ms	remaining: 10.8s
2:	learn: 0.6633070	total: 78.2ms	remaining: 7.74s
3:	learn: 0.6536323	total: 82.3ms	remaining: 6.09s
4:	learn: 0.6448371	total: 86.5ms	remaining: 5.11s
5:	learn: 0.6353840	total: 90.5ms	remaining: 4.43s
6:	learn: 0.6285619	total: 93.8ms	remaining: 3.92s
7:	learn: 0.6210248	total: 96.8ms	remaining: 3.53s
8:	learn: 0.6150294	total: 99.9ms	remaining: 3.23s
9:	learn: 0.6080190	total: 103ms	remaining: 3s
10:	learn: 0.6022094	total: 107ms	remaining: 2.8s
11:	learn: 0.5971392	total: 110ms	remaining: 2.65s
12:	learn: 0.5914814	total: 114ms	remaining: 2.52s
13:	learn: 0.5843863	total: 118ms	remaining: 2.41s
14:	learn: 0.5801079	total: 122ms	remaining: 2.31s
15:	learn: 0.5748371	total: 125ms	remaining: 2.23s
16:	learn: 0.5703197	total: 130ms	remaining: 2.16s
17:	learn: 0.5644419	total: 134ms	remaining: 2.09s
18:	learn: 0.5588265	total: 137ms	remaining: 2.02s
19:	learn: 0.5550034	total: 140ms	re

168:	learn: 0.4244960	total: 652ms	remaining: 505ms
169:	learn: 0.4244000	total: 656ms	remaining: 502ms
170:	learn: 0.4236438	total: 660ms	remaining: 498ms
171:	learn: 0.4234056	total: 665ms	remaining: 495ms
172:	learn: 0.4230349	total: 669ms	remaining: 491ms
173:	learn: 0.4227592	total: 673ms	remaining: 488ms
174:	learn: 0.4223035	total: 680ms	remaining: 486ms
175:	learn: 0.4221212	total: 684ms	remaining: 482ms
176:	learn: 0.4219291	total: 687ms	remaining: 477ms
177:	learn: 0.4216551	total: 690ms	remaining: 473ms
178:	learn: 0.4215603	total: 693ms	remaining: 469ms
179:	learn: 0.4210332	total: 697ms	remaining: 465ms
180:	learn: 0.4198917	total: 700ms	remaining: 460ms
181:	learn: 0.4196638	total: 704ms	remaining: 456ms
182:	learn: 0.4194509	total: 707ms	remaining: 452ms
183:	learn: 0.4190951	total: 710ms	remaining: 448ms
184:	learn: 0.4185814	total: 713ms	remaining: 443ms
185:	learn: 0.4177613	total: 716ms	remaining: 439ms
186:	learn: 0.4173318	total: 720ms	remaining: 435ms
187:	learn: 

63:	learn: 0.4654853	total: 233ms	remaining: 857ms
64:	learn: 0.4645128	total: 237ms	remaining: 856ms
65:	learn: 0.4632976	total: 247ms	remaining: 876ms
66:	learn: 0.4626879	total: 253ms	remaining: 878ms
67:	learn: 0.4619587	total: 256ms	remaining: 874ms
68:	learn: 0.4608741	total: 260ms	remaining: 869ms
69:	learn: 0.4598850	total: 263ms	remaining: 863ms
70:	learn: 0.4594945	total: 266ms	remaining: 858ms
71:	learn: 0.4588876	total: 269ms	remaining: 853ms
72:	learn: 0.4581996	total: 273ms	remaining: 850ms
73:	learn: 0.4574852	total: 277ms	remaining: 847ms
74:	learn: 0.4569101	total: 281ms	remaining: 842ms
75:	learn: 0.4563596	total: 285ms	remaining: 839ms
76:	learn: 0.4551372	total: 289ms	remaining: 836ms
77:	learn: 0.4536966	total: 292ms	remaining: 832ms
78:	learn: 0.4527923	total: 297ms	remaining: 830ms
79:	learn: 0.4523017	total: 300ms	remaining: 824ms
80:	learn: 0.4515843	total: 304ms	remaining: 821ms
81:	learn: 0.4513118	total: 307ms	remaining: 816ms
82:	learn: 0.4509670	total: 310

265:	learn: 0.3967125	total: 1.01s	remaining: 129ms
266:	learn: 0.3964872	total: 1.02s	remaining: 126ms
267:	learn: 0.3958937	total: 1.02s	remaining: 122ms
268:	learn: 0.3958828	total: 1.02s	remaining: 118ms
269:	learn: 0.3955172	total: 1.03s	remaining: 114ms
270:	learn: 0.3955115	total: 1.03s	remaining: 111ms
271:	learn: 0.3945729	total: 1.04s	remaining: 107ms
272:	learn: 0.3945526	total: 1.04s	remaining: 103ms
273:	learn: 0.3942298	total: 1.04s	remaining: 98.9ms
274:	learn: 0.3935090	total: 1.05s	remaining: 95.2ms
275:	learn: 0.3934603	total: 1.05s	remaining: 91.3ms
276:	learn: 0.3934372	total: 1.05s	remaining: 87.4ms
277:	learn: 0.3932649	total: 1.06s	remaining: 83.6ms
278:	learn: 0.3932084	total: 1.06s	remaining: 79.8ms
279:	learn: 0.3930553	total: 1.06s	remaining: 75.9ms
280:	learn: 0.3923420	total: 1.07s	remaining: 72.1ms
281:	learn: 0.3918950	total: 1.07s	remaining: 68.3ms
282:	learn: 0.3916615	total: 1.07s	remaining: 64.5ms
283:	learn: 0.3915988	total: 1.08s	remaining: 60.6ms
2

173:	learn: 0.4170662	total: 627ms	remaining: 454ms
174:	learn: 0.4169832	total: 632ms	remaining: 452ms
175:	learn: 0.4163363	total: 636ms	remaining: 448ms
176:	learn: 0.4159449	total: 640ms	remaining: 445ms
177:	learn: 0.4149943	total: 646ms	remaining: 443ms
178:	learn: 0.4148239	total: 650ms	remaining: 439ms
179:	learn: 0.4143515	total: 653ms	remaining: 435ms
180:	learn: 0.4142922	total: 657ms	remaining: 432ms
181:	learn: 0.4137229	total: 660ms	remaining: 428ms
182:	learn: 0.4135691	total: 663ms	remaining: 424ms
183:	learn: 0.4131581	total: 667ms	remaining: 420ms
184:	learn: 0.4129090	total: 671ms	remaining: 417ms
185:	learn: 0.4119959	total: 675ms	remaining: 414ms
186:	learn: 0.4113864	total: 680ms	remaining: 411ms
187:	learn: 0.4112263	total: 683ms	remaining: 407ms
188:	learn: 0.4109643	total: 686ms	remaining: 403ms
189:	learn: 0.4107920	total: 689ms	remaining: 399ms
190:	learn: 0.4106893	total: 693ms	remaining: 395ms
191:	learn: 0.4104079	total: 696ms	remaining: 392ms
192:	learn: 

...

127:	learn: 0.2032144	total: 22.9s	remaining: 30.8s
128:	learn: 0.2030249	total: 23s	remaining: 30.5s
129:	learn: 0.2028666	total: 23.2s	remaining: 30.3s
130:	learn: 0.2025972	total: 23.3s	remaining: 30.1s
131:	learn: 0.2016190	total: 23.5s	remaining: 29.9s
132:	learn: 0.2008228	total: 23.6s	remaining: 29.7s
133:	learn: 0.2005033	total: 23.8s	remaining: 29.5s
134:	learn: 0.1988783	total: 23.9s	remaining: 29.2s
135:	learn: 0.1974885	total: 24.1s	remaining: 29s
136:	learn: 0.1969934	total: 24.2s	remaining: 28.8s
137:	learn: 0.1961298	total: 24.3s	remaining: 28.6s
138:	learn: 0.1956200	total: 24.5s	remaining: 28.4s
139:	learn: 0.1942988	total: 24.6s	remaining: 28.1s
140:	learn: 0.1936775	total: 24.8s	remaining: 27.9s
141:	learn: 0.1912854	total: 24.9s	remaining: 27.7s
142:	learn: 0.1906997	total: 25.1s	remaining: 27.5s
143:	learn: 0.1903171	total: 25.2s	remaining: 27.3s
144:	learn: 0.1886995	total: 25.4s	remaining: 27.1s
145:	learn: 0.1882988	total: 25.6s	remaining: 27s
146:	learn: 0.1878

287:	learn: 0.1033450	total: 46.6s	remaining: 1.94s
288:	learn: 0.1032086	total: 46.7s	remaining: 1.78s
289:	learn: 0.1031339	total: 46.8s	remaining: 1.61s
290:	learn: 0.1029087	total: 47s	remaining: 1.45s
291:	learn: 0.1027268	total: 47.1s	remaining: 1.29s
292:	learn: 0.1024648	total: 47.3s	remaining: 1.13s
293:	learn: 0.1022281	total: 47.4s	remaining: 968ms
294:	learn: 0.1017587	total: 47.6s	remaining: 807ms
295:	learn: 0.1012737	total: 47.7s	remaining: 645ms
296:	learn: 0.1008358	total: 47.9s	remaining: 484ms
297:	learn: 0.1006792	total: 48s	remaining: 322ms
298:	learn: 0.1002986	total: 48.2s	remaining: 161ms
299:	learn: 0.1001975	total: 48.3s	remaining: 0us
0:	learn: 0.6384854	total: 144ms	remaining: 43.1s
1:	learn: 0.5986482	total: 285ms	remaining: 42.5s
2:	learn: 0.5655921	total: 404ms	remaining: 40s
3:	learn: 0.5360988	total: 574ms	remaining: 42.5s
4:	learn: 0.5119670	total: 716ms	remaining: 42.2s
5:	learn: 0.4955276	total: 854ms	remaining: 41.9s
6:	learn: 0.4842365	total: 1s	re

149:	learn: 0.1648217	total: 20.7s	remaining: 20.7s
150:	learn: 0.1633113	total: 20.9s	remaining: 20.6s
151:	learn: 0.1620276	total: 21s	remaining: 20.5s
152:	learn: 0.1610348	total: 21.2s	remaining: 20.3s
153:	learn: 0.1600658	total: 21.3s	remaining: 20.2s
154:	learn: 0.1593883	total: 21.5s	remaining: 20.1s
155:	learn: 0.1583379	total: 21.6s	remaining: 19.9s
156:	learn: 0.1573895	total: 21.7s	remaining: 19.8s
157:	learn: 0.1560541	total: 21.9s	remaining: 19.7s
158:	learn: 0.1546059	total: 22.1s	remaining: 19.6s
159:	learn: 0.1536627	total: 22.2s	remaining: 19.4s
160:	learn: 0.1521947	total: 22.3s	remaining: 19.3s
161:	learn: 0.1516548	total: 22.5s	remaining: 19.1s
162:	learn: 0.1514166	total: 22.6s	remaining: 19s
163:	learn: 0.1511104	total: 22.8s	remaining: 18.9s
164:	learn: 0.1503493	total: 22.9s	remaining: 18.7s
165:	learn: 0.1500465	total: 23s	remaining: 18.6s
166:	learn: 0.1494911	total: 23.2s	remaining: 18.5s
167:	learn: 0.1475071	total: 23.3s	remaining: 18.3s
168:	learn: 0.1465

18:	learn: 0.5546008	total: 94.5ms	remaining: 1.4s
19:	learn: 0.5509559	total: 99.1ms	remaining: 1.39s
20:	learn: 0.5482205	total: 104ms	remaining: 1.38s
21:	learn: 0.5437778	total: 108ms	remaining: 1.36s
22:	learn: 0.5402969	total: 113ms	remaining: 1.36s
23:	learn: 0.5366279	total: 118ms	remaining: 1.35s
24:	learn: 0.5333881	total: 122ms	remaining: 1.34s
25:	learn: 0.5305079	total: 127ms	remaining: 1.34s
26:	learn: 0.5285267	total: 132ms	remaining: 1.33s
27:	learn: 0.5256420	total: 136ms	remaining: 1.32s
28:	learn: 0.5218247	total: 141ms	remaining: 1.32s
29:	learn: 0.5189722	total: 146ms	remaining: 1.32s
30:	learn: 0.5165531	total: 151ms	remaining: 1.31s
31:	learn: 0.5139200	total: 157ms	remaining: 1.32s
32:	learn: 0.5117903	total: 162ms	remaining: 1.31s
33:	learn: 0.5105310	total: 167ms	remaining: 1.31s
34:	learn: 0.5097080	total: 172ms	remaining: 1.3s
35:	learn: 0.5092421	total: 176ms	remaining: 1.29s
36:	learn: 0.5073880	total: 181ms	remaining: 1.28s
37:	learn: 0.5051204	total: 185

180:	learn: 0.4331830	total: 879ms	remaining: 578ms
181:	learn: 0.4328151	total: 887ms	remaining: 575ms
182:	learn: 0.4326761	total: 892ms	remaining: 570ms
183:	learn: 0.4325384	total: 898ms	remaining: 566ms
184:	learn: 0.4323702	total: 905ms	remaining: 562ms
185:	learn: 0.4317952	total: 910ms	remaining: 557ms
186:	learn: 0.4314032	total: 916ms	remaining: 553ms
187:	learn: 0.4311379	total: 922ms	remaining: 549ms
188:	learn: 0.4309045	total: 926ms	remaining: 544ms
189:	learn: 0.4306426	total: 934ms	remaining: 541ms
190:	learn: 0.4301412	total: 939ms	remaining: 536ms
191:	learn: 0.4294450	total: 943ms	remaining: 531ms
192:	learn: 0.4292177	total: 950ms	remaining: 527ms
193:	learn: 0.4289029	total: 956ms	remaining: 522ms
194:	learn: 0.4287827	total: 961ms	remaining: 517ms
195:	learn: 0.4285255	total: 965ms	remaining: 512ms
196:	learn: 0.4283187	total: 970ms	remaining: 507ms
197:	learn: 0.4282297	total: 974ms	remaining: 502ms
198:	learn: 0.4278679	total: 979ms	remaining: 497ms
199:	learn: 

In [11]:
set_test = pd.merge(data,labels_predict,on = "person", how = "right")
personas = set_test["person"]
set_test = set_test.drop(columns=['person'])

In [12]:
preds_kaggle = cb_model.predict(set_test)

In [13]:
#convert into binary values
for i in range(0,len(preds_kaggle)):
    if preds_kaggle[i]>=.5:       # setting threshold to .5
       preds_kaggle[i]=1
    else:  
       preds_kaggle[i]=0

In [14]:
preds_kaggle  = preds_kaggle.tolist()
df_kaggel = pd.DataFrame(preds_kaggle)
df_kaggel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19415 entries, 0 to 19414
Data columns (total 1 columns):
0    19415 non-null float64
dtypes: float64(1)
memory usage: 151.8 KB


In [15]:
df_kaggel["person"] = personas
df_kaggel.rename(columns = {0: 'label'},inplace = True) 
df_kaggel = df_kaggel[['person','label']]

In [16]:
df_kaggel["label"].value_counts()

0.0    13688
1.0     5727
Name: label, dtype: int64

In [17]:
df_kaggel[("label")] = df_kaggel[("label")].astype(int)
df_kaggel.head()

,person,label
0,00091926,0
1,00091a7a,1
2,000ba417,1
3,000e4d9e,0
4,000e619d,0


In [18]:
df_kaggel.to_csv('Resultados_CatBoost.csv',index=False)